### NEW! : ML Prediction Pipeline

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings("ignore")

from category_encoders import TargetEncoder  # target encoding for categorical features
from statsmodels.tsa.stattools import adfuller  # differencing

# Load dataset
df = pd.read_csv("final_mock_data_3.csv")

In [ ]:
def prediction_pipeline(df, numerical_cols, period, target_var, n_pred_periods=4):
    """Takes in a df, prediction period, output predictions"""

    def encode_categorical_resample(df, period, target_var):
        """
        - Removes ID columns, low-correlated columns to target var
        - Target encodes categorical variables
        - Resamples df to user required period

        Output: df with useful columns (categorical encoded & numerical)
        """
        # First handle string columns
        string_cols = df.select_dtypes(include=["object"]).columns
        for col in string_cols:
            if df[col].nunique() > 0.5 * len(
                df
            ):  # concept: if you got a shit ton of unique values, it's probably an ID
                df = df.drop(columns=[col])

        # Then handle numerical columns correlation
        numerical_cols = df.select_dtypes(include=["number"]).columns
        correlation_threshold = (
            0.4  # Higher as multicollinearity and feature relevance are major concerns.
        )
        corr_matrix = df[numerical_cols].corr()
        cols_to_drop = [
            col
            for col in corr_matrix.columns
            if (corr_matrix[col].abs() < correlation_threshold).sum()
        ]
        df = df.drop(columns=cols_to_drop)

        # Initialize target encoder
        target_encoder = TargetEncoder()

        # Apply target encoding to categorical columns
        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
        df[categorical_cols] = target_encoder.fit_transform(
            df[categorical_cols], df[target_var]
        )

        # apply groupby to get the mean/ mode of the target by each group
        try:
            df = df.resample(
                period
            ).agg(
                {
                    col: "mean"
                    if col in categorical_cols
                    else "sum"  # mean vector for features originally categorical, sum for numerical
                    for col in df.columns
                }
            )

            # Check if too many rows became nil after resampling
            nil_ratio = df.isna().sum() / len(df)
            if (nil_ratio > 0.5).any():  # If more than 50% of any column is nil
                raise ValueError(
                    f"Resampling period '{period}' is too granular for the input data frequency. Many rows became nil after resampling, select another PERIOD"
                )

        except Exception as e:
            raise Exception(f"Error during resampling: {str(e)}")

        # Drop columns with a single unique value or constant
        df = df.loc[
            :, df.nunique() > 1
        ]  # logic -- if categorical vector is same for all periods, it doesn't matter

        return df

    def feature_engineering(df, numerical_cols, target_var):
        """
        Manually implement feature engineering on the input DataFrame,

        Perform feature engineering on the input DataFrame, implementing:
        - Log transform on money related fields -- (revenue, ad spend)
        - EWA (Exponential Weighted Average)
        - Moving average
        - Auto differencing (for stationarity)
        - Lagged variables (for time series)

        Args:
            df: DataFrame with additional engineered features as new columns
        """

        dropped_col = df[target_var].copy()
        df = df.drop(columns=[target_var])

        for col in numerical_cols:
            if col != target_var:
                df[f"log_{col}"] = np.log(df[col])  # Log transform
                df[f"ewa_{col}"] = (
                    df[col].ewm(span=12, adjust=False).mean().shift(1)
                )  # Exponential Weighted Average
                df[f"ma_{col}"] = (
                    df[col].rolling(window=12).mean().shift(1)
                )  # Moving Average

        def check_stationarity(series):
            result = adfuller(series.dropna())
            return result[1] < 0.05  # Returns True if stationary

        # Check and apply differencing for each feature (max 2 times)
        diffed_columns = []
        for col in df.columns:
            diff_count = 1
            while (
                not check_stationarity(df[col]) and diff_count < 3
            ):  # avoid over-differencing -> white noise
                df[col] = df[col].diff()
                df[f"{col}_diff{diff_count}"] = df[col]
                diff_count += 1
            if diff_count > 1:  # If differencing was applied
                diffed_columns.append(col)
                ## Special check for TARGET variable
                # if col == TARGET:
                #     TARGET = f'{col}_diff{diff_count-1}'  # Update TARGET with the last diffed version

        # Remove original columns that had differencing applied -- good practice
        df.drop(columns=diffed_columns, inplace=True)

        # Create lagged variables for each column, except for the target
        for col in df.columns:
            df[f"{col}_lag1"] = df[col].shift(1)
            df[f"{col}_lag2"] = df[col].shift(2)

        df[target_var] = dropped_col
        df = (
            df.dropna()
        )  ## Drop rows with NaN values, better to use for moving averages

        return df

    def feature_selection(
        df, target_var, vif_threshold=7, xgb_importance_threshold=0.05
    ):
        """
        Hybrid feature selection combining VIF and XGBoost importance.

        Args:
            df: DataFrame of features
            vif_threshold: Maximum allowed VIF value (default=7)
            xgb_importance_threshold: Minimum feature importance threshold (default=0.01)

        Returns:
            DataFrame with selected features (columns)
        """
        # First pass: VIF-based selection
        dropped_col = df[target_var].copy()
        df = df.drop(columns=[target_var])
        features = df.columns.tolist()

        while len(features) > 1:
            vif_data = pd.DataFrame()
            vif_data["Feature"] = features
            vif_data["VIF"] = [
                variance_inflation_factor(df[features].values, i)
                for i in range(len(features))
            ]

            if vif_data["VIF"].max() <= vif_threshold:
                print(f"\nAll VIF values are below {vif_threshold}")
                break

            highest_vif_feature = vif_data.loc[vif_data["VIF"].idxmax(), "Feature"]
            features.remove(highest_vif_feature)
            # print(f"\nRemoving feature with highest VIF: {highest_vif_feature}")

            if len(features) == 1:
                print("\nOnly one feature remaining")
                break

        print(f"\nRemaining features after VIF selection: {len(features)}")
        print("Selected features:", features)

        df = df[features]
        df[target_var] = dropped_col

        return df

    def prophet_forecast(df, target_var, period, n_pred_periods):
        df = df.reset_index()
        # Prepare data for Prophet
        df_prophet = df.rename(columns={"Date": "ds", target_var: "y"})

        # Initialize Prophet model
        model = Prophet()
        model.fit(df_prophet)

        # Create future dataframe for prediction
        future = model.make_future_dataframe(periods=n_pred_periods, freq=period)  #

        # Make predictions
        forecast = model.predict(future)
        forecast_snippet = forecast[["ds", "yhat"]][-n_pred_periods:]

        return forecast_snippet, model  # can export model (future saving/ testing)

    df = encode_categorical_resample(df, period, target_var)
    df = feature_engineering(df, numerical_cols, target_var)
    df = feature_selection(df, target_var)
    df, model = prophet_forecast(df, target_var, period, n_pred_periods)

    return df, model


df2, acct_model = prediction_pipeline(df, numerical_cols, "M", "new_accounts", 4)
df2, rev_model = prediction_pipeline(df, numerical_cols, "M", "revenue", 4)
df2, adspend_model = prediction_pipeline(df, numerical_cols, "M", "ad_spend", 4)
print(df2.head())

### Walk forward validation to test Prophet

In [ ]:
def walk_forward_validation(
    df, model, target_var, initial_train_size=12, test_window=1
):
    """
    Perform walk-forward validation on a Prophet model to evaluate its predictive accuracy

    Args:
        df: DataFrame containing the time series data
        model: Pre-trained Prophet model
        target_var: Name of the target variable column
        initial_train_size: Initial number of periods to use for training
        test_window: Number of periods to predict in each iteration

    Returns:
        dict: Dictionary containing evaluation metrics and prediction results
    """
    # Initialize lists to store results
    actuals = []
    predictions = []
    dates = []
    rmse_list = []
    mape_list = []

    # Prepare data in Prophet format
    df_prophet = df.copy()
    df_prophet = df_prophet.reset_index()  # Reset index to make Date a column
    df_prophet = df_prophet.rename(columns={"Date": "ds", target_var: "y"})

    # Calculate total iterations for progress bar
    total_iterations = len(df_prophet) - initial_train_size - test_window + 1

    # Perform walk-forward validation with progress bar
    from tqdm import tqdm

    for i in tqdm(
        range(initial_train_size, len(df_prophet) - test_window + 1),
        desc=f"Validating {target_var} model",
        total=total_iterations,
    ):
        # Get test data for current window
        test_data = df_prophet.iloc[i : i + test_window]

        # Make prediction
        future = test_data[["ds"]]
        forecast = model.predict(future)
        y_pred = forecast["yhat"].values[0]
        y_true = test_data["y"].values[0]

        # Store results
        actuals.append(y_true)
        predictions.append(y_pred)
        dates.append(test_data["ds"].values[0])

        # Calculate error metrics
        rmse = np.sqrt(mean_squared_error([y_true], [y_pred]))
        rmse_list.append(rmse)

        # Calculate MAPE (handle division by zero)
        if y_true != 0:
            mape = np.abs((y_true - y_pred) / y_true) * 100
        else:
            mape = np.nan
        mape_list.append(mape)

    # Calculate summary metrics
    avg_rmse = np.mean(rmse_list)
    avg_mape = np.nanmean(mape_list)

    # Return comprehensive results
    return {
        "avg_rmse": avg_rmse,
        "avg_mape": avg_mape,
        "actuals": actuals,
        "predictions": predictions,
        "dates": dates,
        "target_var": target_var,
        "rmse_list": rmse_list,
        "mape_list": mape_list,
    }


# Evaluate models
print("Model Validation Results:")
print("-" * 50)

# Revenue model evaluation
rev_results = walk_forward_validation(df, rev_model, "revenue")
print("\nRevenue Model:")
print(f"RMSE: {rev_results['avg_rmse']:.2f}")
print(f"MAPE: {rev_results['avg_mape']:.2f}%")

# # Ad spend model evaluation
# ad_results = walk_forward_validation(df, adspend_model, 'ad_spend')
# print(f"\nAd Spend Model:")
# print(f"RMSE: {ad_results['avg_rmse']:.2f}")
# print(f"MAPE: {ad_results['avg_mape']:.2f}%")

# # New accounts model evaluation
# acct_results = walk_forward_validation(df, acct_model, 'new_accounts')
# print(f"\nNew Accounts Model:")
# print(f"RMSE: {acct_results['avg_rmse']:.2f}")
# print(f"MAPE: {acct_results['avg_mape']:.2f}%")

# # Plot the results
# import matplotlib.pyplot as plt

# # Create subplots for each metric
# fig, axes = plt.subplots(3, 1, figsize=(15, 12))
# fig.suptitle('Predicted vs Actual Values')

# # Revenue plot
# axes[0].plot(rev_results['dates'], rev_results['actuals'], label='Actual', marker='o')
# axes[0].plot(rev_results['dates'], rev_results['predictions'], label='Predicted', marker='x')
# axes[0].set_title('Revenue')
# axes[0].legend()
# axes[0].grid(True)

# # Ad Spend plot
# axes[1].plot(ad_results['dates'], ad_results['actuals'], label='Actual', marker='o')
# axes[1].plot(ad_results['dates'], ad_results['predictions'], label='Predicted', marker='x')
# axes[1].set_title('Ad Spend')
# axes[1].legend()
# axes[1].grid(True)

# # New Accounts plot
# axes[2].plot(acct_results['dates'], acct_results['actuals'], label='Actual', marker='o')
# axes[2].plot(acct_results['dates'], acct_results['predictions'], label='Predicted', marker='x')
# axes[2].set_title('New Accounts')
# axes[2].legend()
# axes[2].grid(True)

# plt.tight_layout()
# plt.show()

### Archive (addition to feature selection -- XGBoost hybrid)

In [ ]:
# # Second pass: XGBoost-based selection
# X = df[features]
# y = df[TARGET]

# # Train XGBoost model
# xgb_model = XGBRegressor(random_state=42)
# xgb_model.fit(X, y)

# # Get feature importance
# importance_df = pd.DataFrame({
#     'Feature': features,
#     'Importance': xgb_model.feature_importances_
# })

# print("\nXGBoost Feature Importance:")
# print(importance_df.sort_values('Importance', ascending=False))

# # Select features based on importance threshold
# selected_features = importance_df[importance_df['Importance'] >= xgb_importance_threshold]['Feature'].tolist()

# # selected_features.append(target_col)  # Add back target column (make sure added back properly)

# print(f"\nSelected {len(selected_features)-1} features based on XGBoost importance")

In [ ]:
# print(X_filtered)

# # Split Data
# X_train, X_test, y_train, y_test = train_test_split(X_filtered, df['target'], test_size=0.2, random_state=42)

# # Train XGBoost for Feature Importance
# xgb_model = XGBRegressor()
# xgb_model.fit(X_train, y_train)
# feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': xgb_model.feature_importances_})
# top_features = feature_importance.sort_values(by='Importance', ascending=False).head(5)['Feature'].tolist()
# X_train, X_test = X_train[top_features], X_test[top_features]

# # Train Prophet Model
# df_prophet = df[['target']].reset_index().rename(columns={'date': 'ds', 'target': 'y'})
# prophet = Prophet()
# prophet.fit(df_prophet)
# future = prophet.make_future_dataframe(periods=len(y_test))
# prophet_forecast = prophet.predict(future)['yhat'].tail(len(y_test)).values

# # Train ARIMAX Model
# arimax = SARIMAX(y_train, exog=X_train, order=(1,1,1))
# arimax_fit = arimax.fit()
# arimax_forecast = arimax_fit.forecast(steps=len(y_test), exog=X_test)

# # Train XGBoost Model
# xgb_forecast = xgb_model.predict(X_test)

# # Train LSTM Model
# scaler = MinMaxScaler()
# y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1,1))
# y_test_scaled = scaler.transform(y_test.values.reshape(-1,1))
# lstm_model = Sequential([
#     LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)),
#     Dropout(0.2),
#     LSTM(50, activation='relu'),
#     Dropout(0.2),
#     Dense(1)
# ])
# lstm_model.compile(optimizer='adam', loss='mse')
# lstm_model.fit(np.expand_dims(X_train, axis=2), y_train_scaled, epochs=50, batch_size=16, verbose=0)
# lstm_forecast = scaler.inverse_transform(lstm_model.predict(np.expand_dims(X_test, axis=2)))

# # Stacking Model (Meta-Learner)
# stacking_features = np.column_stack([
#     prophet_forecast,
#     arimax_forecast,
#     xgb_forecast,
#     lstm_forecast.flatten()
# ])
# meta_model = LinearRegression()
# meta_model.fit(stacking_features, y_test)
# stacked_forecast = meta_model.predict(stacking_features)

# # Evaluate Models
# def evaluate(y_true, y_pred):
#     return {
#         'MAE': mean_absolute_error(y_true, y_pred),
#         'RMSE': np.sqrt(mean_squared_error(y_true, y_pred))
#     }

# evaluations = {
#     'Prophet': evaluate(y_test, prophet_forecast),
#     'ARIMAX': evaluate(y_test, arimax_forecast),
#     'XGBoost': evaluate(y_test, xgb_forecast),
#     'LSTM': evaluate(y_test, lstm_forecast.flatten()),
#     'Stacked Model': evaluate(y_test, stacked_forecast)
# }

# # Select Best Model
# best_model = min(evaluations, key=lambda x: evaluations[x]['RMSE'])
# print("Best Model:", best_model)
# print("Evaluation Scores:", evaluations)

### Archive: Old function to combine into single DF

In [ ]:
def prophet(from_date, to_date):
    df = pd.read_csv("final_mock_data.csv")
    df["Date"] = pd.to_datetime(df["Date"])
    df = df[df["Date"].isin(pd.date_range(start=from_date, end=to_date))]
    time_period = 4
    # takes in df_grouped, makes df_plotting

    df_grouped = (
        df.resample("M", on="Date")
        .agg(
            {
                "ad_spend": "sum",
                "new_accounts": "sum",
                "revenue": "sum",
            }
        )
        .reset_index()
    )

    df_plotting = df_grouped[["Date", "revenue", "ad_spend", "new_accounts"]]
    df_plotting.rename(columns={"Date": "ds"}, inplace=True)

    def prophet_forecast(df_plotting, df_grouped, predict_col, time_period):
        # Prepare data for Prophet
        df_prophet = df_grouped.rename(columns={"Date": "ds", predict_col: "y"})
        df_grouped.rename(
            columns={"Date": "ds"}, inplace=True
        )  # for subsequent merging

        # Initialize Prophet model
        model = Prophet()
        model.fit(df_prophet)

        # Create future dataframe for prediction
        future = model.make_future_dataframe(periods=time_period, freq="M")

        # Make predictions
        forecast = model.predict(future)
        forecast_snippet = forecast[["ds", "yhat"]][-time_period:]

        # Add future months to df_plotting
        last_date = df_grouped["ds"].max()
        future_dates = pd.date_range(
            start=last_date + pd.DateOffset(months=1), periods=time_period, freq="M"
        )
        future_df = pd.DataFrame({"ds": future_dates})

        # Only add empty values for columns that don't already exist
        for col in ["revenue", "ad_spend", "new_accounts"]:
            if col not in future_df.columns:
                future_df[col] = np.nan

        df_plotting = pd.concat([df_plotting, future_df], ignore_index=True)

        # Update predictions in df_plotting
        for idx, row in forecast_snippet.iterrows():
            df_plotting.loc[df_plotting["ds"] == row["ds"], predict_col] = row["yhat"]

        return df_plotting

    # Get individual forecasts
    df_plotting_rev = prophet_forecast(df_plotting, df_grouped, "revenue", 4)
    df_plotting_ad = prophet_forecast(df_plotting, df_grouped, "ad_spend", 4)
    df_plotting_accounts = prophet_forecast(df_plotting, df_grouped, "new_accounts", 4)

    # Combine the forecasts
    df_plotting_combined = pd.DataFrame()
    df_plotting_combined["ds"] = df_plotting_rev["ds"]
    df_plotting_combined["revenue"] = df_plotting_rev["revenue"]
    df_plotting_combined["ad_spend"] = df_plotting_ad["ad_spend"]
    df_plotting_combined["new_accounts"] = df_plotting_accounts["new_accounts"]

    df_plotting = df_plotting_combined

    return df_plotting


df2 = prophet("2022-01-01", "2024-09-30")
print(df2)